In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")

text = text[:100000]


Taille du vocabulaire : 6745


In [2]:
my_input_sequences = []
for line in text.split('. '):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)

max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

X = input_sequences[:, :-1]  
y = input_sequences[:, -1]  
y = np.array(tf.keras.utils.to_categorical(y, num_classes=vocab_size))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(Bidirectional(GRU(120)))
model.add(Dense(vocab_size, activation='softmax'))

model.build(input_shape=(None, X.shape[1])) 


In [3]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

print(model.summary())



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 120, 50)        │       337,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 240)            │       123,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6745)           │     1,625,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,086,635 (7.96 MB)

 Trainable params: 2,086,635 (7.96 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=32,
    callbacks=[early_stopping]
)



Epoch 1/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.0293 - loss: 7.1179 - val_accuracy: 0.0622 - val_loss: 6.2424
Epoch 2/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.0691 - loss: 5.8063 - val_accuracy: 0.0832 - val_loss: 6.0410
Epoch 3/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.0946 - loss: 5.3913 - val_accuracy: 0.1011 - val_loss: 5.9589
Epoch 4/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1165 - loss: 4.9807 - val_accuracy: 0.1057 - val_loss: 5.9677
Epoch 5/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1326 - loss: 4.6609 - val_accuracy: 0.1025 - val_loss: 6.0383
Epoch 6/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1572 - loss: 4.3517 - val_accuracy: 0.1130 - val_loss: 6.0927
Epoch 7/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.1759 - loss: 4.0502 - val_accuracy: 0.1136 - val_loss: 6.1975
Epoch 8/80
428/428 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.2085 - loss: 3.7696 - 

In [5]:
input_text = "But i want to talk"
predict_next_words = 50

for _ in range(predict_next_words):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

print(input_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━